<a href="https://colab.research.google.com/github/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting/blob/main/notebooks/n_beats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import userdata
token = userdata.get('GITHUB_TOKEN')
user_name = userdata.get('GITHUB_USERNAME')
mail = userdata.get('GITHUB_MAIL')

!git config --global user.name "{user_name}"
!git config --global user.email "{mail}"
!git clone https://{token}@github.com/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting.git

%cd Walmart-Recruiting-Store-Sales-Forecasting

from google.colab import userdata
! pip install -r ./requirements.txt
kaggle_json_path = userdata.get('KAGGLE_JSON_PATH')
! ./src/data_loader.sh -f {kaggle_json_path}

Mounted at /content/drive
Cloning into 'Walmart-Recruiting-Store-Sales-Forecasting'...
remote: Enumerating objects: 326, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 326 (delta 59), reused 48 (delta 21), pack-reused 209 (from 1)
Receiving objects: 100% (326/326), 6.89 MiB | 25.58 MiB/s, done.
Resolving deltas: 100% (156/156), done.
/content/Walmart-Recruiting-Store-Sales-Forecasting
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 28.1

In [164]:
# **Torch**

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.config import *

stores = pd.read_csv(STORES_PATH)
features = pd.read_csv(FEATURES_PATH)
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

train['Date'] = pd.to_datetime(train.Date)

In [165]:
from src.processing import split_data
x_train,y_train,x_val,y_val = split_data(train)

**Cross Validation**

In [166]:
class CustomNBEATS(NBEATS):
    def configure_optimizers(self):
        # Define your custom optimizer here
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)

        # Optional: add scheduler if needed
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

        return {
            'optimizer': optimizer,
            # 'lr_scheduler': {
            #     'scheduler': scheduler,
            #     'interval': 'epoch',
            #     'frequency': 1
            # }
        }

In [195]:
from sklearn.base import BaseEstimator, TransformerMixin

class Preprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        x_copy = X.copy()
        if 'y' not in x_copy.columns:
            x_copy['y'] = 0
        return x_copy


In [196]:
from sklearn.base import BaseEstimator, TransformerMixin
import torch
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
from neuralforecast.losses.pytorch import MSE

class NBEATSModel(BaseEstimator, TransformerMixin):
    def __init__(self,models:list[NBEATS]=None):
        if models == None or len(models)==0:
          models = [CustomNBEATS(
            input_size=15,
            h=48,
            max_steps=25 * 104,
            batch_size=64,
            stack_types=(['identity', 'trend', 'seasonality']),
            n_blocks = [1,1,1],
            random_seed=42,
            accelerator='gpu',
            devices=1,
            logger=True,
            enable_progress_bar=False,
            enable_model_summary=False,
        )]
        self.nf = NeuralForecast(models=models, freq='W-FRI')

    def fit(self, X, y=None):
        df = self.create_data_nbeats(X,y)
        df.sort_values(by=['ds'],inplace=True)
        self.nf.fit(df)

        return self

    def transform(self, X):
        preds = self.nf.predict()

        real = self.create_data_nbeats(X,X.y)
        merged = pd.merge(real, preds, on=['unique_id','ds'],how='left')
        # print(merged.shape)
        merged.fillna(0,inplace=True)


        return merged


    def predict(self,X=None):
        return self.transform(X)

    def create_data_nbeats(self,x:pd.DataFrame,y:pd.DataFrame) -> pd.DataFrame:
        df = pd.DataFrame()

        df = df.reset_index()
        df["Date"] = x.Date
        df["Store"] = x.Store
        df["Dept"] = x.Dept
        df["IsHoliday"] = x.IsHoliday.astype(int)
        df.rename(columns={"Date": "ds"}, inplace=True)
        df["unique_id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str)
        df["y"] = y.values
        df = df[["unique_id", "ds", "y",'IsHoliday']].copy()
        return df

In [197]:
from os import pipe
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('preprocess', Preprocessor()),
    ('model', NBEATSModel())
])

model = pipeline.fit(x_train,y_train)

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.


In [199]:
from src.utils import wmae
x_val['y'] = y_val
preds = model.predict(x_val)
wmae(preds['y'],preds['CustomNBEATS'],preds['IsHoliday'])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


np.float64(2462.0331187605916)

In [175]:
test.Date = pd.to_datetime(test.Date)

In [200]:
model.predict(test)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


,unique_id,ds,y,IsHoliday,CustomNBEATS
0,1_1,2012-11-02,0,0,0.0
1,1_1,2012-11-09,0,0,0.0
2,1_1,2012-11-16,0,0,0.0
3,1_1,2012-11-23,0,1,0.0
4,1_1,2012-11-30,0,0,0.0
...,...,...,...,...,...
115059,45_98,2013-06-28,0,0,0.0
115060,45_98,2013-07-05,0,0,0.0
115061,45_98,2013-07-12,0,0,0.0
115062,45_98,2013-07-19,0,0,0.0
